# Genomic Sequencing - Week 4

### How do we sequence antibiotics (Part II)?

We have successfully reconstructed the Tyrocidine B1 peptide given its ideal spectrum. Realistically mass spectrometers produce noisy spectra that are not ideal usually having **false masses** and **missing masses**. A false mass if present in the experimental spectrum but absent from the theoretical spectrum; a missing mass is present in the theoretical spectrum but absent from the experimental spectrum.

Example Theoretical and Experimental Spectrum of NQEL:<br>
<img src="img/theoretical_simulated_spectrum_nqel.png" width="600">

Any false or missing masses cause the current CyclopeptideSequencing function to throw out the correct peptide. How can we reformulate the function to handle experimental spectra with errors?

We need to relax the requirement that a candidates theoretical spectrum must match the experimental one exactly. A **scoring function** that will select the peptide whose theoretical spectrum matches a given experimental one the most closely.
Given a cyclic peptide and a spectrum, Score(peptide, spectrum) is the number of masses shared between cyclospectrum(peptide) and spectrum.

In [3]:
# Functions from previous Weeks
mass_table = "../Week3/Data/integer_mass_table.txt"

intMass = {}
massInt = {}
with open(mass_table) as inFile:
    for key, mass in [x.strip().split(" ") for x in inFile.readlines()]:
        intMass[key] = int(mass)
        massInt[int(mass)] = key
    massInt[113] = 'I'
    massInt[128] = 'K'

def calcMass(peptide, total=True, ext=False):
    if total:
        if ext:
            return(sum([ord(x) for x in peptide]))
        else:
            return sum([intMass[x] for x in peptide])
    else:
        return [intMass[x] for x in peptide]

def linearSpectrum(peptide, isMassArray=False, isExt=False):
    prefMass = [0]
    for i in range(len(peptide)):
        if isMassArray:
            prefMass.append(prefMass[i] + peptide[i])
        elif isExt:
            prefMass.append(prefMass[i] + ord(peptide[i]))
        else:
            prefMass.append(prefMass[i] + intMass[peptide[i]])
    linSpec = [0]
    for i in range(len(prefMass)):
        for j in range(i + 1, len(prefMass)):
            linSpec.append(prefMass[j] - prefMass[i])
    return sorted(linSpec)

def cyclicSpectrum(peptide, isMassArray=False, isExt=False, prnt=False, ret=True):       
    prefMass = [0]
    for i in range(len(peptide)):
        if isMassArray:
            prefMass.append(prefMass[i] + peptide[i])
        elif isExt:
            prefMass.append(prefMass[i] + ord(peptide[i]))
        else:
            prefMass.append(prefMass[i] + intMass[peptide[i]])
    pepMass = prefMass[len(peptide)]
    cycSpec = [0]
    for i in range(len(prefMass)):
        for j in range(i+1, len(prefMass)):
            cycSpec.append(prefMass[j] - prefMass[i])
            if i > 0 and j < len(peptide):
                cycSpec.append(pepMass - cycSpec[-1])
    cycSpec = sorted(cycSpec)
    if prnt:
        print(*cycSpec, sep=" ")
    if ret:
        return cycSpec
    
def pepToMassString(peptide):
    return '-'.join([str(intMass[amino]) for amino in peptide])

In [6]:
def score(tSpec, spectrum):
    '''
    Score a theotritcal spectrum against the experimental spectrum.
    Return number of masses that overlap between the given throretical spectrum and experimental spectrum.
    
    Args:
        tSpec (list[int]): Linear or Cyclic theoretical spectrum of a peptide.
        spectrum (list[int]): Experimental spectrum of the peptide of interest.
        
    Returns:
        (int): Number of overlapping masses between the two spectrums.
    '''
    res = []
    temp = spectrum[:]
    for mass in tSpec[:]:
        if mass in temp:
            res.append(mass)
            tSpec.remove(mass)
            temp.remove(mass)
    return len(res)

In [5]:
# test/run score function

# spec = [0, 99, 113, 114, 128, 227, 257, 299, 355, 356, 370, 371, 484]
# score('NQEL', spec)

# with open("./Data/dataset_102_3.txt") as inFile:
with open("./Data/dataset_4913_1.txt") as inFile:
    data = inFile.readlines()

pep = data[0].strip()
spec = [int(x) for x in data[1].strip().split(' ')]
score(linearSpectrum(pep), spec)

260

Redefining our problem to Cyclopeptide Sequencing for spectra with errors. Since the original function had a strict bounding step and removed all candidate linear peptides having inconsistent spectra. We should revise to include more candidate linear peptides without getting out of control. Utilizing a leaderboard to hold the N highest scoring candidates at each iteration and extend those.

In [7]:
def trim(leadBoard, spectrum, N, isMassArray=False, isExt=False):
    '''
    Trim list of candidate peptides to only the top results.
    Peptides are scored based on linearSpectrum.
    The Score for the Nth peptide (when sorted from largest to smallest score) is saved
        and all peptides with this score or higher are returned.
        
    Args:
        leadBoard (list[string] or list[list[int]]): List of candidate peptides in string or mass array format.
        spectrum (list[int]): Experimental spectrum of the peptide of interest.
        N (int): Cutoff index for returned candidates.
        isMassArray (bool, optional): Defaults to False. If True the given leaderboard is treated as a list of mass arrays.
        isExt (bool, optional): Defualts to False. If True the peptides are generated from an extended amino acid table.
        
    Returns:
        (list[string] or list[list[int]]): List of candidate peptides that have a linear spectrum with a score >= score(N).
    '''
    N -= 1
    if len(leadBoard) <= N:
        return leadBoard
    linScore = []
    for pep in leadBoard:
        linScore.append((pep, score(linearSpectrum(pep, isMassArray=isMassArray, isExt=isExt), spectrum)))
    linScore = sorted(linScore, key=lambda x:x[1], reverse=True)
    tScore = linScore[N][1]
    i = N + 1
    while i < len(linScore) and linScore[i][1] >= tScore:
        i +=1
    return [x[0] for x in linScore[:i]]

In [8]:
# test/run trim function

# trim(['LAST', 'ALST', 'TLLT', 'TQAS'], [0, 71, 87, 101, 113, 158, 184, 188, 259, 271, 372], 2)

with open("Data/dataset_4913_3.txt") as inFile:
    data = inFile.readlines()
    lBoard = [x for x in data[0].strip().split(' ')]
    spec = [int(x) for x in data[1].strip().split(' ')]
    n = int(data[2].strip())
print(*trim(lBoard, spec, n), sep=' ')

ADLSLPWNVLGCCHRQQGDIYMDQQPLTPEKRPQWG MCDMRSGTYKVPKFSYIQDTVRHPVEMVKFYGAAMR GTNNTIPAIIVRALDQAVCTSTICWMVWSQQWCGDV CAWVCPFLDVEGRDCGNFWGGCLWYLHRQGCNPLEY KRFRNMEPDQSIFRRWNYIDDPIAIGNSHFMIRGMP


In [10]:
aa = ['G', 'A', 'S', 'P', 'V', 'T', 'C', 'I', 'N', 'D', 'K', 'E', 'M', 'H', 'F', 'R', 'Y', 'W']
masses = [57, 71, 87, 97, 99, 101, 103, 113, 114, 115, 128, 129, 131, 137, 147, 156, 163, 186]

def leaderCyclopeptideSequencing(Spectrum, N):
    '''
    Find the most likely peptide that could produce the given experimental mass spectrum.
    
    Args:
        Spectrum (list[int]): Experimental spectrum of the peptide of interest.
        N (int): Cutoff index for returned candidates.
        
    Returns:
        (string and list[strings]): Latest peptide to achieve a max cyclical spectrum score compared to the
            experimental spectrum as well as a list of all candidate peptides that had the same max score.
    '''
    parentMass = max(Spectrum)
    leaderPeptides = []
    leader = None
    maxCycloScore = 0
    truePeptides = ['']
    while truePeptides:
        leaderboard = [pept + amino for pept in truePeptides for amino in aa]
        truePeptides = []
        for candidate in leaderboard:
            candidateMass = calcMass(candidate)
            if candidateMass == parentMass:
                candidateCycloScore = score(cyclicSpectrum(candidate), Spectrum)
                if candidateCycloScore >= maxCycloScore:
                    if candidateCycloScore > maxCycloScore:
                        leader = candidate
                        maxCycloScore = candidateCycloScore
                        leaderPeptides = [candidate]
                    else:
                        leaderPeptides.append(candidate)
            if candidateMass <= parentMass:
                truePeptides.append(candidate)
        truePeptides = trim(truePeptides, Spectrum, N)
    print(f"Max CycloScore: {maxCycloScore}\n# of candiate linear peptides: {len(leaderPeptides)}")
    return leader, leaderPeptides

In [7]:
%%time
# test/run leaderCycloPepSeq function

# spec = [0, 71, 113, 129, 147, 200, 218, 260, 313, 331, 347, 389, 460]
# test, leads = leaderCyclopeptideSequencing(spec, 10)
# print(pepToMassString(test))

# with open("Data/dataset_102_8.txt") as inFile:
#     data = inFile.readlines()
# n = int(data[0].strip())
# spec = [int(x) for x in data[1].strip().split(' ')]
# test, leads = leaderCyclopeptideSequencing(spec, n)
# st = ''
# for i, a in enumerate(test):
#     if i != 0:
#         st += '-'
#     st += str(intMass[a])
# print(st)

# with open("Data/Tyrocidine_B1_Spectrum_25.txt") as inFile:
with open("Data/dataset_102_10.txt") as inFile:
    data = inFile.readlines()
    data = [int(x) for x in data[1].strip().split(' ')]
leader, leadList = leaderCyclopeptideSequencing(data, 1000)
cycSet = set()
for lead in leadList:
    st = ''
    cy = cyclicSpectrum(lead)
    for i, mass in enumerate(cy):
        if i != 0:
            st += '-'
        st += str(mass)
    cycSet.add(st)
print(len(cycSet))
# print(*cycSet, sep=' ')

Max CycloScore: 83
# of candiate linear peptides: 38
14
Wall time: 5.38 s


So far we have considered just 20 amino acids that form the building blocks of proteins called **proteinogenic amino acids**. There are actually two more called **selenocysteine** and **pyrrolysine**. In addition to the 22 protinogenic, NRPs contain **non-proteinogenic amino acids**, expanding the possible number of building blocks for antibiotic peptides from 20 to over 100.

In [17]:
def leaderCyclopeptideSequencingExt(Spectrum, N, aa):
    '''
    Find the most likely peptide that could produce the given experimental mass spectrum constructed from the given set
        of possible amino acids.
        
    Args:
        Spectrum (list[int]): Experimental spectrum of the peptide of interest.
        N (int): Cutoff index for returned candidates.
        aa (list[string]): Set of amino acids that can be utilized to find potential candidate peptides.
        
    Returns:
        (string and list[strings]): Latest peptide to achieve a max cyclical spectrum score compared to the
            experimental spectrum as well as a list of all candidate peptides that had the same max score.
    '''
    parentMass = max(Spectrum)
    leaderPeptides = []
    leader = None
    maxCycloScore = 0
    truePeptides = ['']
    while truePeptides:
        leaderboard = [pept + amino for pept in truePeptides for amino in aa]
        truePeptides = []
        for candidate in leaderboard:
            candidateMass = calcMass(candidate, ext=True)
            if candidateMass == parentMass:
                candidateCycloScore = score(cyclicSpectrum(candidate, isExt=True), Spectrum)
                if candidateCycloScore >= maxCycloScore:
                    if candidateCycloScore > maxCycloScore:
                        leader = candidate
                        maxCycloScore = candidateCycloScore
                        leaderPeptides = [candidate]
                    else:
                        leaderPeptides.append(candidate)
            if candidateMass <= parentMass:
                truePeptides.append(candidate)
        truePeptides = trim(truePeptides, Spectrum, N, isExt=True)
    print(f"Max CycloScore: {maxCycloScore}\n# of candiate linear peptides: {len(leaderPeptides)}")
    return leader, leaderPeptides

In [18]:
%%time
# test/run leaderCyclopeptideSequencingExt function

aa = [chr(i) for i in range(57, 201)]
with open("Data/Tyrocidine_B1_Spectrum_10.txt") as inFile:
    data = inFile.readlines()
spec = [int(x) for x in data[0].strip().split(' ')]
lead, leadList = leaderCyclopeptideSequencingExt(spec, 1000, aa)
print(len(leadList))

Max CycloScore: 87
# of candiate linear peptides: 34
34
Wall time: 53.4 s


In [19]:
linPeps = []
for pep in leadList:
    st = ''
    for i, amino in enumerate(pep):
        if i != 0:
            st += '-'
        st += str(ord(amino))
    linPeps.append(st)
print(*linPeps, sep=' ')

113-147-97-186-147-114-128-98-65-99-128 147-97-186-147-114-128-98-65-99-128-113 128-113-147-97-186-147-114-128-98-65-99 114-147-186-97-147-113-128-99-65-98-128 128-99-163-128-114-147-113-73-97-147-113 128-114-147-186-97-147-113-128-99-65-98 147-186-97-147-113-128-99-65-98-128-114 99-163-128-114-147-124-62-97-147-113-128 147-114-128-163-99-128-113-147-97-73-113 97-186-147-114-128-98-65-99-128-113-147 99-163-128-114-147-113-73-97-147-113-128 99-163-128-114-147-186-97-72-75-113-128 113-147-97-186-147-114-128-98-65-99-63-65 113-147-97-186-147-114-128-98-65-99-57-71 113-147-97-186-147-114-128-98-65-99-58-70 113-147-97-186-147-114-128-98-65-99-59-69 113-147-97-186-147-114-128-98-65-99-60-68 113-147-97-186-147-114-128-98-65-99-61-67 113-147-97-186-147-114-128-98-65-99-62-66 113-147-97-186-147-114-128-98-65-99-64-64 113-147-97-186-147-114-128-98-65-99-65-63 113-147-97-186-147-114-128-98-65-99-66-62 113-147-97-186-147-114-128-98-65-99-67-61 113-147-97-186-147-114-128-98-65-99-68-60 113-147-97-1

Given only an experimental spectrum how could you elucidate the amino acid composition? **Convolution** of a spectrum is the process of taking all positive differences of the masses in the spectrum.
<img src="./img/spectral_convolution_nqel.png" width="450">

In [20]:
def spectralConvolution(spec):
    '''
    Calcualte the convolution of a given experimetnal spectrum.
    Convolution = all positive differences of masses in the spectrum.
    
    Args:
        spec (list[int]): Experimental spectrum of the peptide of interest.
    
    Returns:
        (list[int]): List of all positive mass differences within the experimental spectrum.
    '''
    res = []
    for i in range(len(spec)):
        for x in range(len(spec)):
            if i != x:
                mass = spec[i] - spec[x]
                if mass > 0:
                    res.append(mass)
    return res

In [21]:
spec = [0, 137, 186, 323]
spectralConvolution(spec)

# with open("Data/dataset_104_4.txt") as inFile:
#     data = inFile.readlines()
#     spec = [int(x) for x in data[0].strip().split(' ')]
# print(*spectralConvolution(spec), sep=' ')

[137, 186, 49, 323, 186, 137]

In [35]:
def convolutionCyclopeptideSequencing(M, N, spec):
    '''
    Find the most likely peptide that could produce the given experimental mass spectrum constructed from the
        most likely masses within the spectrum convolution.
        
    Args:
        M (int): Number of top elements to keep from the spectrum convolution.
        N (int): Cutoff index for returned candidates.
        spec (list[int]): Experimental spectrum of the peptide of interest.
    
    Returns:
        (string and list[strings]): Latest peptide to achieve a max cyclical spectrum score compared to the
            experimental spectrum as well as a list of all candidate peptides that had the same max score.
    '''
    convoDict = {}
    spec = sorted(spec)
    convo = spectralConvolution(spec)
    for mass in convo:
        if mass not in convoDict:
            convoDict[mass] = 1
        else:
            convoDict[mass] += 1
    convoDict = sorted(list(convoDict.items()), key=lambda x: x[1], reverse=True)
    aa = [chr(mass) for mass, freq in convoDict if 57 <= mass <= 200 and freq >= 2]
    return leaderCyclopeptideSequencingExt(spec, N, aa)

In [36]:
%%time
spec = [57, 57, 71, 99, 129, 137, 170, 186, 194, 208, 228, 265, 285, 299, 307, 323, 356, 364, 394, 422, 493]
lead, leaders = convolutionCyclopeptideSequencing(20, 60, spec)

# with open("Data/dataset_104_7.txt") as inFile:
#     data = inFile.readlines()
#     M = int(data[0].strip())
#     N = int(data[1].strip())
#     spec = [int(x) for x in data[2].strip().split(' ')]

# with open("Data/Tyrocidine_B1_Spectrum_25.txt") as inFile:
#     data = inFile.readlines()
#     spec = [int(x) for x in data[0].strip().split(' ')]
# convolutionCyclopeptideSequencing(20, 1000, spec)
    
# # convolutionCyclopeptideSequencing(M, N, spec)

st = ''
for i, aa in enumerate(lead):
    if i != 0:
        st += '-'
    st += str(ord(aa))
print(st)

Max CycloScore: 21
# of candiate linear peptides: 8
99-71-137-57-57-72
Wall time: 74 ms


###### Quiz

In [25]:
# 3
print(f"#3 Cyclic Score of MAMA: {score(cyclicSpectrum('MAMA'), [0, 71, 178, 202, 202, 202, 333, 333, 333, 404, 507, 507])}")

# 4
print(f"#4 Linear Score of PEEP: {score(linearSpectrum('PEEP'), [0, 97, 97, 129, 194, 196, 226, 226, 244, 258, 323, 323, 452])}")

# 5
specConv = spectralConvolution([0, 57, 118, 179, 236, 240, 301])
convDict = {}
for mass in specConv:
    if mass not in convDict:
        convDict[mass] = 1
    else:
        convDict[mass] += 1
print(f"#5 Element with highest multiplicity: {sorted(list(convDict.items()), key=lambda x: x[1], reverse=True)[0][0]}")

#3 Cyclic Score of MAMA: 8
#4 Linear Score of PEEP: 8
#5 Element with highest multiplicity: 61


###### Bioinformatics Application Challenge

In [1]:
def ngStatCalc(contigs, N=0.5, totalLen=None):
    '''
    Calculate NXX, NGXX given a collection of contigs and N.
    
    Args:
        contigs (list[int]): Collection of contig lengths.
        N (int, optional): Default to 0.5. Fraction to determine (ex. N50 = 0.5, N75 = 0.75).
        totalLen (int, optional): Default to None. Total length of genome for NGXX calcualtion.
        
    Returns:
        (int): Length of contig in which all contigs of the same length or greater represent
            N percentage of the total nucleotides.
    '''
    contigs = sorted(contigs, reverse=True)
    i = 0
    track = contigs[i]
    if not totalLen:
        totalLen = sum(contigs)
#     print(f"totalLen: {totalLen}")
    while track/totalLen < N:
        i += 1
        track += contigs[i]
    return contigs[i]

In [5]:
# Compute N50 and N75 for the following contigs
contigs = [20,20,30,30,60,60,80,100,200]
print(f"N50: {ngStatCalc(contigs)}")
print(f"N75: {ngStatCalc([20,20,30,30,60,60,80,100,200], N=0.75)}")
print(f"NG50 with length 1000: {ngStatCalc(contigs, totalLen=1000)}")
print(f"NGA50: {ngStatCalc([20,20,30,30,50,50,60,60,80,200], totalLen=1000)}")

N50: 100
N75: 60
NG50 with length 1000: 60
NGA50: 50
